In [ ]:
from math import sin, cos
import random
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from Farm_Evaluator_grasp import getTurbLoc, loadPowerCurve, binWindResourceData, preProcessing, preProcessing_greedy, getAEP, getAEP_greedy, getAEP_greedy_all_years, getAEP_all_years, checkConstraints


In [ ]:
def gen_coord():
    x = round(random.uniform(min_lim, max_lim),2)
    y = round(random.uniform(min_lim, max_lim),2)
    return x,y

In [ ]:
# Problem parameter
min_lim = 50
max_lim = 3950
turb_count = 50
penalty = 0.02

In [ ]:
from shapely import affinity
from shapely.geometry.point import Point

def create_ellipse(center, lengths, angle):
    circ = Point(center).buffer(1)
    ell = affinity.scale(circ, int(lengths[0]), int(lengths[1]))
    ellr = affinity.rotate(ell, angle)
    return ellr

major = 225
minor = 200
angle = 60

def check_violation(turb1, turb2):
    '''
    Return True if intersection is empty
    '''
    ellipse1 = create_ellipse(turb1, (major,minor), angle)
    ellipse2 = create_ellipse(turb2, (major,minor), angle)
    intersect = ellipse1.intersection(ellipse2)
    return intersect.is_empty

In [ ]:
def generate_turb_locations(count):
    arr = [gen_coord()]
    for i in range(count-1):
        while True:
            new_cord = gen_coord()

            if all([check_violation(np.array(cord), np.array(new_cord)) for cord in arr]):
                arr.append(new_cord)
                break
    return arr

def count_violation(turb_coords):
    violation_count = 0
    for i,turb1 in enumerate(turb_coords):
        for turb2 in np.delete(turb_coords, i, axis=0):
            if  np.linalg.norm(turb1 - turb2) < 8*turb_rad:
                violation_count += 1
    return violation_count

In [ ]:
num_turbines = 1

# Turbine Specifications.
# -**-SHOULD NOT BE MODIFIED-**-
turb_specs    =  {   
                     'Name': 'Anon Name',
                     'Vendor': 'Anon Vendor',
                     'Type': 'Anon Type',
                     'Dia (m)': 100,
                     'Rotor Area (m2)': 7853,
                     'Hub Height (m)': 100,
                     'Cut-in Wind Speed (m/s)': 3.5,
                     'Cut-out Wind Speed (m/s)': 25,
                     'Rated Wind Speed (m/s)': 15,
                     'Rated Power (MW)': 3
                 }
turb_diam      =  turb_specs['Dia (m)']
turb_rad       =  turb_diam/2 

# Turbine x,y coordinates
#turb_coords   =  getTurbLoc('turbine_loc_test.csv')

# Load the power curve
power_curve   =  loadPowerCurve('power_curve.csv')

# Pass wind data csv file location to function binWindResourceData.
# Retrieve probabilities of wind instance occurence.
#wind_inst_freq =  binWindResourceData('wind_data_2007.csv')   

'''wind_data = [
'wind_data_2007.csv',
'wind_data_2008.csv',
'wind_data_2009.csv',
'wind_data_2013.csv',
'wind_data_2014.csv',
'wind_data_2015.csv',
'wind_data_2017.csv'
]'''


wind_data = [
'wind_data_2009.csv'
]



# Doing preprocessing to avoid the same repeating calculations. Record 
# the required data for calculations. Do that once. Data are set up (shaped)
# to assist vectorization. Used later in function totalAEP.
n_wind_instances, cos_dir, sin_dir, wind_sped_stacked, C_t = preProcessing_greedy(power_curve, num_turbines)

def greedy_randomized_solution(alpha):
    width = 3900
    height = 3900
    n_turbs = 50
    points = np.array(generate_turb_locations(n_turbs))
    for i in points:
        i[0] = i[0] + 50
        i[1] = i[1] + 50
    solution = np.array([[]])
    solution = np.array([], dtype=np.int64).reshape(0,2)
    best_value = 0
    best_point = 0
    x = np.array([[]])
    for i in range(n_turbs):
        best_value = 0
        eval_list = []
        for j in range(len(points)):
            objec = points[j]
            if(objec not in solution):
                solution = np.vstack((solution, objec))
                obj = getAEP_greedy_all_years(turb_rad, solution, power_curve, wind_data, n_wind_instances, cos_dir, sin_dir, wind_sped_stacked, C_t) 
                eval_list.append((j, obj))
                if(obj >= best_value):
                    best_value = obj
                    best_point = objec
                solution = np.delete(solution, len(solution) - 1, 0)
        sorted_vec = sorted(eval_list, key=lambda tup: tup[1], reverse=True)
        positions = int((len(sorted_vec)-1)*alpha)
        pos  = random.randint(0, positions)
        point = points[sorted_vec[pos][0]]
        solution = np.vstack((solution, point))
    neighbors = np.array(list(filter(lambda x: x not in solution, points)))
    return solution, points


def greedy_solution():
    width = 3900
    height = 3900
    n_turbs = 50
    points = np.array(generate_turb_locations(n_turbs)).flatten()
    for i in points:
        i[0] = i[0] + 50
        i[1] = i[1] + 50
    solution = np.array([[]])
    solution = np.array([], dtype=np.int64).reshape(0,2)
    best_value = 0
    best_point = 0
    x = np.array([[]])
    for i in range(n_turbs):
        best_value = 0
        for j in range(len(points)):
            objec = points[j]
            if(objec not in solution):
                solution = np.vstack((solution, objec))
                obj = getAEP_greedy_all_years(turb_rad, solution, power_curve, wind_data, n_wind_instances, cos_dir, sin_dir, wind_sped_stacked, C_t) 
                if(obj >= best_value):
                    best_value = obj
                    best_point = objec
                solution = np.delete(solution, len(solution) - 1, 0)
        solution = np.vstack((solution, best_point))
    neighbors = np.array(list(filter(lambda x: x not in solution, points)))
    return solution

def perturb(solution, neighbors, position, best_value):
    best_point = solution[position]
    print("Initial ", best_value)
    for i in neighbors:
        solution[position] = i
        obj = getAEP_all_years(turb_rad, solution, power_curve, wind_data, n_wind_instances, cos_dir, sin_dir, wind_sped_stacked, C_t) 
        if(obj >= best_value):
            best_value = obj
            best_point = i
    solution[position] = best_point
    print("Later ", best_value)

    return solution, best_value

def local_search(solution, points):
    best_solution = solution.copy()
    neighbors = np.array(list(filter(lambda x: x not in solution, points)))
    best_value = getAEP_all_years(turb_rad, solution, power_curve, wind_data, n_wind_instances, cos_dir, sin_dir, wind_sped_stacked, C_t) 
    for position in range(n_turbs):
        solution = best_solution.copy()
        improved = False
        for i in neighbors:
            solution[position] = i.copy()
            value = getAEP_all_years(turb_rad, solution, power_curve, wind_data, n_wind_instances, cos_dir, sin_dir, wind_sped_stacked, C_t) 
            if(value >= best_value):
                best_value = value
                best_point = i
                improved = True
                best_solution = solution.copy()
        neighbors = np.array(list(filter(lambda x: x not in best_solution, points)))
    return best_solution, best_value

def grasp(max_iter, alpha):
    best_solution = np.array([[]])
    best_solution = np.array([], dtype=np.int64).reshape(0,2)
    best_value = 0
    for position in range(max_iter):
        solution, points = greedy_randomized_solution(alpha)
        solution, value = local_search(solution, points)
        print(value, best_value)
        if(value >= best_value):
            best_value = value
            best_solution = solution.copy()
            print("Best value in iteration no {}: {}".format(position, best_value))
            print(best_value)
    return best_solution, best_value


n_turbs = 50
alpha = 0.6

a, b = grasp(10, alpha)

#sol1 = greedy_randomized_solution(alpha)
#best_solution = getAEP_all_years(turb_rad, sol1, power_curve, wind_inst_freq, n_wind_instances, cos_dir, sin_dir, wind_sped_stacked, C_t) 
#checkConstraints(sol1, turb_diam)
#print(best_solution)

# sol2 = greedy_solution()
# best_solution = getAEP_all_years(turb_rad, sol2, power_curve, wind_inst_freq, n_wind_instances, cos_dir, sin_dir, wind_sped_stacked, C_t) 
# checkConstraints(sol2, turb_diam)
# print(best_solution)

# for i in range(50):
#     if(sol1[i] in sol2):
#         print("tem")
#     else:
#         print("não tem")

# n_wind_instances, cos_dir, sin_dir, wind_sped_stacked, C_t = preProcessing(power_curve)
# solu = local_search(sol)
# best_solution = getAEP_all_years(turb_rad, solu, power_curve, wind_inst_freq, n_wind_instances, cos_dir, sin_dir, wind_sped_stacked, C_t) 
# print(best_solution)
